# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
weather_data = "../output_data/cities.csv"
weatherdf = pd.read_csv(weather_data)
weatherdf = weatherdf.dropna()
weatherdf.head()

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,maple creek,49.9168,-109.4848,-2.15,71,100,9.22,CA,1643743772
1,itacoatiara,-3.1431,-58.4442,85.15,68,91,4.56,BR,1643743772
2,santa rosa,14.3122,121.1114,76.86,75,25,10.18,PH,1643743188
3,rikitea,-23.1203,-134.9692,78.04,70,100,7.25,PF,1643743772
4,poopo,-18.3667,-66.9667,54.37,59,62,4.54,BO,1643743773


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
# Access maps with unique API key
gmaps.configure(api_key=g_key)

locations = weatherdf[['Lat','Lng']].astype(float)

humidity = weatherdf["Humidity"].astype(float)

# Create a humidity Heatmap layer
fig = gmaps.figure()

heat_layer = gmaps.heatmap_layer(locations, weights=humidity, 
                                 dissipating=False, max_intensity=100,
                                 point_radius = 1)

fig.add_layer(heat_layer)

fig


Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [7]:
weatherdf = weatherdf.dropna()

weatherdfvaca = weatherdf.drop(weatherdf.index[(weatherdf['Humidity']>70 )])
weatherdfvaca = weatherdfvaca.drop(weatherdfvaca.index[(weatherdfvaca['Max Temp'] < 65)])
weatherdfvaca = weatherdfvaca.drop(weatherdfvaca.index[(weatherdfvaca['Max Temp'] > 85)])
# weatherdfvaca = weatherdfvaca.drop(weatherdfvaca.index[(weatherdfvaca['Wind Speed']<20)])

# weatherdfvaca.head() 
weatherdfvaca.count()

City          68
Lat           68
Lng           68
Max Temp      68
Humidity      68
Cloudiness    68
Wind Speed    68
Country       68
Date          68
dtype: int64

### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [8]:
hotel_df = weatherdfvaca

hotel_df["Hotel Name"] = ''

# geocoordinates
target_coordinates =  hotel_df[['Lat','Lng']].astype(float)
# target_search = "Hotel"
target_radius = 5000
target_type = "hotel"

# set up a parameters dictionary
params = {
    "location": target_coordinates,
#     "keyword": target_search,
    "radius": target_radius,
    "type": target_type,
    "key": g_key
}

# base url
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

# run a request using our params dictionary
response = requests.get(base_url, params=params)

In [9]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [10]:
# Add marker layer ontop of heat map
hotel_layer = gmaps.symbol_layer(
    locations, fill_color='rgba(0, 150, 0, 0.4)',
    stroke_color='rgba(0, 0, 150, 0.4)', scale=2,
    
)


fig = gmaps.figure()
fig.add_layer(hotel_layer)

fig

# Display figure


Figure(layout=FigureLayout(height='420px'))